# Kepler Framework, Emulate v11

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency
from Gaugi import mkdir_p, progressbar
import numpy as np
import pandas as pd
import collections
import os
from pprint import pprint
from copy import deepcopy

import gc
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


In [2]:
output_path = 'output/emulation'
mkdir_p(output_path)

In [3]:
dpath_data17 = '../../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_data18 = '../../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'
dpath_mc16   = '../../data/output/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.dataframe.h5'

In [4]:
tpath_data17 = 'output/emulation/data17_table_v11.h5'
tpath_data18 = 'output/emulation/data18_table_v11.h5'
tpath_mc16   = 'output/emulation/mc16_table_v11.h5'

## Setup Chains:

In [5]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI", L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v11_vloose'),
            Chain( "HLT_e28_lhtight_nod0_ringer_v11_ivarloose", L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v11_tight' ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI", L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v11_medium'),
            Chain( "HLT_e140_lhloose_nod0_ringer_v11" , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_v11_loose'),
]

def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'ringer_v11_vloose')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'ringer_v11_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'ringer_v11_medium')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'ringer_v11_loose')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])


## Emulate 2017:

Take something like 1 hour

In [6]:
data17_df = pd.concat( (load_hdf(dpath_data17), load_hdf(tpath_data17)), axis=1)

In [7]:
emulate(data17_df)

Emulate...: 100%|██████████| 4/4 [01:49<00:00, 27.36s/it]


In [8]:
data17_df.shape

(43311283, 111)

### Validate:

In [9]:
eff_data17_Zee = Efficiency( output_path+'/data17_Zee_efficiency_v11.root')

2022-03-12 22:13:17,023 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_Zee_efficiency_v11.root


In [10]:
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data17_Zee.fill( data17_df.loc[data17_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:53<00:00, 22.62s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1069  | 30158981/32047563 |
| L2Calo | 93.4322  | 29942745/32047563 |
|   L2   | 93.1923  | 29865855/32047563 |
| EFCalo | 91.8823  | 29446027/32047563 |
|  HLT   | 89.9816  | 28836896/32047563 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:21<00:00, 16.36s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6937  | 21557594/22765602 |
| L2Calo | 94.2486  | 21456272/22765602 |
|   L2   | 94.1328  | 21429904/22765602 |
| EFCalo | 93.2073  | 21219199/22765602 |
|  HLT   | 87.0336  | 19813726/22765602 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5079  | 1361055/1367786 |
| L2Calo | 97.1517  | 1328827/1367786 |
|   L2   | 97.0612  | 1327590/1367786 |
| EFCalo | 94.9685  | 1298966/1367786 |
|  HLT   | 91.7130  | 1254437/1367786 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.50it/s]


HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 100.0000 | 60541/60541  |
| L2Calo | 97.9353  | 59291/60541  |
|   L2   | 97.8659  | 59249/60541  |
| EFCalo | 96.5445  | 58449/60541  |
|  HLT   | 94.5904  | 57266/60541  |
+--------+----------+--------------+


In [11]:
eff_data17_Zee.save()

In [12]:
eff_data17_JF17 = Efficiency( output_path+'/data17_JF17_efficiency_v11.root')

2022-03-12 22:17:38,271 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data17_JF17_efficiency_v11.root


In [13]:
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data17_JF17.fill( data17_df.loc[data17_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:22<00:00,  4.41s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+------------------+
|  Step  | Eff [%%] |   passed/total   |
+--------+----------+------------------+
| L1Calo | 84.4275  | 8738241/10349992 |
| L2Calo |  3.1103  | 321915/10349992  |
|   L2   |  2.7271  | 282253/10349992  |
| EFCalo |  2.5617  | 265131/10349992  |
|  HLT   |  0.1317  |  13632/10349992  |
+--------+----------+------------------+


Filling...: 100%|██████████| 5/5 [00:19<00:00,  3.94s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 90.6060  | 8384948/9254300 |
| L2Calo |  0.5128  |  47452/9254300  |
|   L2   |  0.4684  |  43349/9254300  |
| EFCalo |  0.3581  |  33138/9254300  |
|  HLT   |  0.0013  |   119/9254300   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9012  | 7844422/7852181 |
| L2Calo |  0.5468  |  42939/7852181  |
|   L2   |  0.4932  |  38724/7852181  |
| EFCalo |  0.3508  |  27542/7852181  |
|  HLT   |  0.0015  |   115/7852181   |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:12<00:00,  2.48s/it]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9227  | 5629684/5634041 |
| L2Calo |  2.5647  |  144496/5634041 |
|   L2   |  2.2599  |  127324/5634041 |
| EFCalo |  2.0106  |  113279/5634041 |
|  HLT   |  0.0404  |   2275/5634041  |
+--------+----------+-----------------+


In [14]:
eff_data17_JF17.save()

In [15]:
del data17_df

## Emulate 2018:

In [16]:
data18_df = pd.concat( (load_hdf(dpath_data18), load_hdf(tpath_data18)), axis=1)

In [17]:
emulate(data18_df)

Emulate...: 100%|██████████| 4/4 [01:33<00:00, 23.29s/it]


In [18]:
data18_df.shape

(41873973, 111)

In [19]:
eff_data18_Zee = Efficiency( output_path+'/data18_Zee_efficiency_v11.root')

2022-03-12 22:22:19,491 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_Zee_efficiency_v11.root


In [20]:
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_data18_Zee.fill( data18_df.loc[data18_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [01:52<00:00, 22.42s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.1301  | 30705693/32620495 |
| L2Calo | 93.4730  | 30491348/32620495 |
|   L2   | 93.2368  | 30414316/32620495 |
| EFCalo | 91.9045  | 29979702/32620495 |
|  HLT   | 90.4817  | 29515590/32620495 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [01:23<00:00, 16.64s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-------------------+
|  Step  | Eff [%%] |    passed/total   |
+--------+----------+-------------------+
| L1Calo | 94.6555  | 22067825/23313826 |
| L2Calo | 94.2124  | 21964512/23313826 |
|   L2   | 94.1162  | 21942094/23313826 |
| EFCalo | 93.3161  | 21755547/23313826 |
|  HLT   | 87.5987  | 20422618/23313826 |
+--------+----------+-------------------+


Filling...: 100%|██████████| 5/5 [00:06<00:00,  1.20s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.5563  | 1375545/1381675 |
| L2Calo | 97.1634  | 1342483/1381675 |
|   L2   | 97.0746  | 1341255/1381675 |
| EFCalo | 94.9854  | 1312389/1381675 |
|  HLT   | 92.0252  | 1271489/1381675 |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 20.20it/s]


HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9934  | 60385/60389  |
| L2Calo | 97.9003  | 59121/60389  |
|   L2   | 97.8026  | 59062/60389  |
| EFCalo | 96.6219  | 58349/60389  |
|  HLT   | 94.8169  | 57259/60389  |
+--------+----------+--------------+


In [21]:
eff_data18_Zee.save()

In [22]:
eff_data18_JF17 = Efficiency( output_path+'/data18_JF17_efficiency_v11.root')

2022-03-12 22:26:35,235 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/data18_JF17_efficiency_v11.root


In [23]:
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='!el_lhvloose',reset=True)
eff_data18_JF17.fill( data18_df.loc[data18_df['target']==0], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='!el_lhvloose',reset=True)

Filling...: 100%|██████████| 5/5 [00:17<00:00,  3.58s/it]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 85.5183  | 7240922/8467098 |
| L2Calo |  3.5932  |  304241/8467098 |
|   L2   |  3.1459  |  266367/8467098 |
| EFCalo |  2.9818  |  252474/8467098 |
|  HLT   |  0.5780  |  48936/8467098  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:15<00:00,  3.19s/it]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 91.4008  | 6972786/7628804 |
| L2Calo |  0.8220  |  62712/7628804  |
|   L2   |  0.7797  |  59483/7628804  |
| EFCalo |  0.6734  |  51373/7628804  |
|  HLT   |  0.0392  |   2994/7628804  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:13<00:00,  2.73s/it]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.8995  | 6555389/6561985 |
| L2Calo |  0.8273  |  54285/6561985  |
|   L2   |  0.7758  |  50907/6561985  |
| EFCalo |  0.6409  |  42057/6561985  |
|  HLT   |  0.0468  |   3073/6561985  |
+--------+----------+-----------------+


Filling...: 100%|██████████| 5/5 [00:10<00:00,  2.07s/it]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+-----------------+
|  Step  | Eff [%%] |   passed/total  |
+--------+----------+-----------------+
| L1Calo | 99.9159  | 4772502/4776518 |
| L2Calo |  3.0427  |  145337/4776518 |
|   L2   |  2.7018  |  129054/4776518 |
| EFCalo |  2.4595  |  117480/4776518 |
|  HLT   |  0.3110  |  14853/4776518  |
+--------+----------+-----------------+


In [24]:
eff_data18_JF17.save()

## Evaluate mc16 boosted:

In [25]:
mc16_df = pd.concat( (load_hdf(dpath_mc16), load_hdf(tpath_mc16)), axis=1)

In [26]:
emulate(mc16_df)

Emulate...: 100%|██████████| 4/4 [00:00<00:00, 28.39it/s]


In [27]:
eff_mc16 = Efficiency( output_path+'/mc16_boosted_efficiency_v11.root')

2022-03-12 22:27:52,007 | Py.Efficiency                           INFO Creating the StoreGate service with path: output/emulation/mc16_boosted_efficiency_v11.root


In [28]:
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI', pidname='el_lhvloose', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e28_lhtight_nod0_ringer_v11_ivarloose' , pidname='el_lhtight' , reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI', pidname='el_lhmedium', reset=True)
eff_mc16.fill( mc16_df.loc[mc16_df['target']==1], 'HLT_e140_lhloose_nod0_ringer_v11'          , pidname='el_lhloose' , reset=True)

Filling...: 100%|██████████| 5/5 [00:00<00:00, 16.06it/s]


HLT_e17_lhvloose_nod0_ringer_v11_L1EM15VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.7303  | 78407/78619  |
| L2Calo | 96.9104  | 76190/78619  |
|   L2   | 96.9002  | 76182/78619  |
| EFCalo | 76.2297  | 59931/78619  |
|  HLT   | 75.1650  | 59094/78619  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.41it/s]


HLT_e28_lhtight_nod0_ringer_v11_ivarloose
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.8427  | 72990/73105  |
| L2Calo | 89.3031  | 65285/73105  |
|   L2   | 89.2976  | 65281/73105  |
| EFCalo | 71.4383  | 52225/73105  |
|  HLT   | 68.7901  | 50289/73105  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 17.14it/s]


HLT_e60_lhmedium_nod0_ringer_v11_L1EM24VHI
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9635  | 73911/73938  |
| L2Calo | 92.7047  | 68544/73938  |
|   L2   | 92.6993  | 68540/73938  |
| EFCalo | 73.1045  | 54052/73938  |
|  HLT   | 71.3517  | 52756/73938  |
+--------+----------+--------------+


Filling...: 100%|██████████| 5/5 [00:00<00:00, 18.17it/s]

HLT_e140_lhloose_nod0_ringer_v11
+--------+----------+--------------+
|  Step  | Eff [%%] | passed/total |
+--------+----------+--------------+
| L1Calo | 99.9882  | 67947/67955  |
| L2Calo | 96.7022  | 65714/67955  |
|   L2   | 96.6978  | 65711/67955  |
| EFCalo | 75.4234  | 51254/67955  |
|  HLT   | 73.9798  | 50273/67955  |
+--------+----------+--------------+


In [29]:
eff_mc16.save()